In [1]:


base_dir = '../../../../T5 EVO'

In [2]:
# # import sys
# # sys.path
# # sys.path.append('/path/to/the/example_file.py')
import sys
sys.path.append("/Volumes/work/phd/‌MoLFormer_fMRI")

In [3]:
! ls '../'
# !ls '/Volumes'

243_22562_220-pycharm-support-libs output_layers.csv
41597_2021_1033_MOESM1_ESM.asv     random_methods.py
NoteBooks                          read_ev.ipynb
constants.py                       results
extract_representations.ipynb      smiles_cid_subject.csv
mat_to_niftii.m                    spm12
odor_names.xlsx                    utils
odor_responses_S1-3_regionized


In [4]:
! ls '/Volumes/Macintosh HD'

/Volumes/Macintosh HD


In [5]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.metrics import mean_squared_error
import scipy
from sklearn.decomposition import PCA
from utils.helpers import *
from scipy import stats
import numpy as np
import pandas as pd
seed = 2024

In [6]:
def custom_ridge_regression(X, y,alpha):
    if alpha is None:
        linreg = RidgeCV(alphas=np.logspace(2,7,16),cv=5)
    else:
        linreg = Ridge(alpha=alpha)
    # linreg = MultiOutputRegressor(linreg,n_jobs=-1)

    estimator = linreg.fit(X, y)
    return estimator

In [7]:
def metrics_per_descriptor(X, y, linreg):
 
    """
    Calculate predictions, mean squared errors, and Pearson correlations for each descriptor.

    Parameters:
    X (array-like): Input features.
    y (array-like): True target values.
    linreg (object): Linear regression model with a predict method.

    Returns:
    tuple: A tuple containing predicted values, a list of MSEs, and a list of correlations.
    """
    predicted = linreg.predict(X)
    predicted = predicted if len(predicted.shape) > 1 else predicted.reshape(-1, 1)
    
    # Ensure y is 2D for consistent iteration
    y = y if len(y.shape) > 1 else y.reshape(-1, 1)

    mse_errors = [mean_squared_error(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]


    np.random.shuffle(y)
    mse_errors_shuffle = [mean_squared_error(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    spearman_correlations_shuffle = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
    return predicted, mse_errors, correlations,spearman_correlations, mse_errors_shuffle, correlations_shuffle,spearman_correlations_shuffle

In [8]:
def train_and_eval_ntimes_alphas(X,y,voxels_retained, times,n_components=None):
    """
    Train and evaluate a regression model using cross-validation.

    Parameters:
    data (DataFrame): Input data containing embeddings and fmri_average.
    times (int): Number of cross-validation iterations.
    n_components (int, optional): Number of components for dimensionality reduction.

    Returns:
    tuple: Contains CIDs, predicted values, test targets, runs, MSE errors, and correlations.
    """
    mse_errors_crossvalidated = []
    correlations_crossvalidated = []
    spearman_crossvalidated = []


    mse_errors_shuffle_crossvalidated = []
    correlations_shuffle_crossvalidated = []
    spearman_shuffle_crossvalidated = []


    predicteds = []
    y_tests = []
    runs_predictions = []
    runs_metrics = []
    voxels = []
    voxels_predicteds = []
    alphas = np.logspace(2,7,16)
    for i in range(times):
        correlations_per_alpha = []
        print(f"Processing run {i}")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=seed + i
        )
        X_test, X_val, y_test, y_val = train_test_split(
            X_test, y_test, test_size=0.5, random_state=seed + i
        )
        if n_components:
            pca = PCA(n_components=n_components)
            X_train = pca.fit_transform(X_train)
            X_val = pca.fit_transform(X_val)
            X_test = pca.transform(X_test)

        for alpha in alphas:
            linreg = custom_ridge_regression(X_train, y_train,alpha)
            predicted, mse_errors, correlations,spearman_correlation, mse_errors_shuffle, correlations_shuffle,spearman_correlations_shuffle = metrics_per_descriptor(X_val, y_val, linreg)
            correlations = np.asarray(correlations)
            correlations_per_alpha.append(correlations[:,0])
        if len(correlations_per_alpha) == 0:
            #create empty arrays
            predicted = np.zeros((y_test.shape[0],y_test.shape[1]))
            mse_errors = np.zeros(y_test.shape[1])
            correlations = np.zeros((y_test.shape[1],2))
            spearman_correlation = np.zeros((y_test.shape[1],2))

            mse_errors_shuffle = np.zeros(y_test.shape[1])
            correlations_shuffle = np.zeros((y_test.shape[1],2))
            spearman_correlations_shuffle = np.zeros((y_test.shape[1],2))


        elif len(correlations_per_alpha) == 1:
            best_alpha = alphas[0]
            linreg = custom_ridge_regression(X_train, y_train,best_alpha)
            predicted, mse_errors, correlations,spearman_correlation, mse_errors_shuffle, correlations_shuffle,spearman_correlations_shuffle = metrics_per_descriptor(X_test, y_test, linreg)
        else:
            idx_best_alpha = np.argmax(np.mean(correlations_per_alpha, axis=1))
            best_alpha = alphas[idx_best_alpha]
            linreg = custom_ridge_regression(X_train, y_train,best_alpha)
            predicted, mse_errors, correlations,spearman_correlation, mse_errors_shuffle, correlations_shuffle,spearman_correlations_shuffle = metrics_per_descriptor(X_test, y_test, linreg)





        #flatten the predicted and y_test arrays
        voxels_predicted = np.arange(predicted.shape[1])
        predicted = np.concatenate(predicted)
        y_test_flatten = np.concatenate(y_test)
        mse_errors_crossvalidated.extend(mse_errors)
        correlations_crossvalidated.extend(correlations)
        spearman_crossvalidated.extend(spearman_correlation)
        predicteds.extend(predicted)
        y_tests.extend(y_test_flatten)

        runs_metrics.extend([i] * y_test.shape[1])
        runs_predictions.extend([i] * y_test.shape[0]*y_test.shape[1])

        voxels.extend(voxels_retained)

        voxels_predicted = np.repeat(voxels_predicted,y_test.shape[0])
        voxels_predicteds.extend(voxels_predicted)


        mse_errors_shuffle_crossvalidated.extend(mse_errors_shuffle)
        correlations_shuffle_crossvalidated.extend(correlations_shuffle)
        spearman_shuffle_crossvalidated.extend(spearman_correlations_shuffle)

    return predicteds, y_tests, runs_metrics,runs_predictions,\
        mse_errors_crossvalidated, correlations_crossvalidated,spearman_crossvalidated,\
        mse_errors_shuffle_crossvalidated,correlations_shuffle_crossvalidated,spearman_shuffle_crossvalidated,\
        voxels,voxels_predicteds

In [9]:
def train_and_eval_ntimes(X,y, voxels_retained,times,n_components=None):
    """
    Train and evaluate a regression model using cross-validation.

    Parameters:
    data (DataFrame): Input data containing embeddings and fmri_average.
    times (int): Number of cross-validation iterations.
    n_components (int, optional): Number of components for dimensionality reduction.

    Returns:
    tuple: Contains CIDs, predicted values, test targets, runs, MSE errors, and correlations.
    """
    mse_errors_crossvalidated = []
    correlations_crossvalidated = []
    spearman_crossvalidated = []


    mse_errors_shuffle_crossvalidated = []
    correlations_shuffle_crossvalidated = []
    spearman_shuffle_crossvalidated = []


    predicteds = []
    y_tests = []
    runs_predictions = []
    runs_metrics = []
    voxels = []
    voxels_predicteds = []
    # alphas = np.logspace(2,7,16)
    for i in range(times):
        correlations_per_alpha = []
        print(f"Processing run {i}")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=seed + i
        )

        if n_components:
            pca = PCA(n_components=n_components)
            X_train = pca.fit_transform(X_train)
            X_test = pca.transform(X_test)

        # for alpha in alphas:
        linreg = custom_ridge_regression(X_train, y_train,None)






        predicted, mse_errors, correlations,spearman_correlation, mse_errors_shuffle, correlations_shuffle,spearman_correlations_shuffle = metrics_per_descriptor(X_test, y_test, linreg)


        if voxels_retained is None:
            voxels_retained = range(y_train.shape[1])


        #flatten the predicted and y_test arrays
        voxels_predicted = np.arange(predicted.shape[1])
        predicted = np.concatenate(predicted)
        y_test_flatten = np.concatenate(y_test)
        mse_errors_crossvalidated.extend(mse_errors)
        correlations_crossvalidated.extend(correlations)
        spearman_crossvalidated.extend(spearman_correlation)
        predicteds.extend(predicted)
        y_tests.extend(y_test_flatten)

        runs_metrics.extend([i] * y_test.shape[1])
        runs_predictions.extend([i] * y_test.shape[0]*y_test.shape[1])

        voxels.extend(voxels_retained)

        voxels_predicted = np.repeat(voxels_predicted,y_test.shape[0])
        voxels_predicteds.extend(voxels_predicted)


        mse_errors_shuffle_crossvalidated.extend(mse_errors_shuffle)
        correlations_shuffle_crossvalidated.extend(correlations_shuffle)
        spearman_shuffle_crossvalidated.extend(spearman_correlations_shuffle)

    return predicteds, y_tests, runs_metrics,runs_predictions,\
        mse_errors_crossvalidated, correlations_crossvalidated,spearman_crossvalidated,\
        mse_errors_shuffle_crossvalidated,correlations_shuffle_crossvalidated,spearman_shuffle_crossvalidated,\
        voxels,voxels_predicteds


In [10]:
def pipeline(embeddings,fmri,voxels_retained, times=30,n_components=None,learning_type='cvridge'):
    """
    Run the pipeline for a specific voxel and model, processing fMRI and embeddings data.

    Parameters:
    fmri_ev (DataFrame): fMRI event data.
    voxel (int): The voxel identifier.
    model_name (str): Name of the model being evaluated.
    input_file (str): Path to the input CSV file.
    times (int, optional): Number of cross-validation iterations. Default is 30.
    n_components (int, optional): Number of components for dimensionality reduction. Default is None.

    Returns:
    tuple: DataFrames containing predictions and metrics.
    """
    # Filter voxel-specific data
    y = fmri
    X = embeddings
    
    

    # Train and evaluate the model
    if learning_type == 'ridge':
        results = train_and_eval_ntimes_alphas(X,y,voxels_retained, times=times,n_components=n_components)
    elif learning_type=='cvridge':
        results = train_and_eval_ntimes(X,y,voxels_retained, times=times,n_components=n_components)

    (predicteds, y_tests, runs_metrics,runs_predictions,
     mse_errors, correlations,spearman_correlations,
     mse_errors_shuffle, correlations_shuffle,spearman_correlations_shuffle,
     voxel,voxels_predicteds) = results
    if len(predicteds) == 0:
        return None,None
    predicteds = np.asarray(predicteds)
    y_tests = np.asarray(y_tests)
   
    runs_metrics = np.asarray(runs_metrics)
    runs_predictions = np.asarray(runs_predictions)
    mse_errors = np.asarray(mse_errors)
    correlations = np.asarray(correlations)
    spearman_correlations = np.asarray(spearman_correlations)

    mse_errors_shuffle = np.asarray(mse_errors_shuffle)
    correlations_shuffle = np.asarray(correlations_shuffle)
    spearman_correlations_shuffle = np.asarray(spearman_correlations_shuffle)

    voxels = np.asarray(voxel)
    voxels_predicteds = np.asarray(voxels_predicteds)




    stats_correlations = correlations[:, 0]
    pvalues_correlations = correlations[:, 1]
    
    stats_spearman_correlations = spearman_correlations[:, 0]
    pvalues_spearman_correlations = spearman_correlations[:, 1]


    stats_correlations_shuffle = correlations_shuffle[:, 0]
    pvalues_correlations_shuffle = correlations_shuffle[:, 1]

    stats_spearman_correlations_shuffle = spearman_correlations_shuffle[:, 0]
    pvalues_spearman_correlations_shuffle = spearman_correlations_shuffle[:, 1]
    

    
    metrics_df = pd.DataFrame(
        np.column_stack([stats_correlations, pvalues_correlations, mse_errors,voxels,stats_spearman_correlations,pvalues_spearman_correlations,runs_metrics]),
        columns=['pearson_correlation', 'pearson_pvalue', 'mse','voxel','spearman_correlation','spearman_pvalue','run']
    )

    metrics_shuffle = pd.DataFrame(
        np.column_stack([stats_correlations_shuffle, pvalues_correlations_shuffle, mse_errors_shuffle,voxels,stats_spearman_correlations_shuffle,pvalues_spearman_correlations_shuffle,runs_metrics]),
        columns=['pearson_correlation', 'pearson_pvalue', 'mse','voxel','spearman_correlation','spearman_pvalue','run']
    )
    
    predictions_df = pd.DataFrame(
        np.column_stack([predicteds, y_tests,voxels_predicteds,runs_predictions]),
        columns=['predicted', 'target', 'voxel','run']
    )

    return predictions_df, metrics_df,metrics_shuffle

In [11]:
def compute_correlation(X,y,voxels_retained=None,n_components=None,learning_type='cvridge'):
    """
    Compute correlations for MolFormer across specified layers and voxels.

    Parameters:
    times (int): Number of cross-validation iterations.
    n_components (int): Number of components for dimensionality reduction.
    fmri_ev (DataFrame): fMRI event data.
    input_file_molformer (str): Base path for MolFormer input files.
    layers (list, optional): List of layers to process. Default is [13].

    Returns:
    tuple: Metrics and predictions for MolFormer.
    """
    metrics = []
    metrics_shuffle = []
    predictions = []

    # create dictionary keys as indices of fmri_all_voxels and values as fmri_all_voxels
    # data_dict = {'voxel_'+str(i):y[:,i] for i in range(y.shape[1])}
    # data_dict['CID'] = CIDs
    # fmri_df = pd.DataFrame(data_dict)
    
    
        
        
    
    # X: behavior/embeddings, y: fmri
    results = pipeline(X,
        y, voxels_retained,times=15,n_components=n_components,learning_type=learning_type)

    df_predictions, df_metric,df_metric_shuffle = results



    df_metric["subject"]=subject
    df_metric_shuffle["subject"]=subject
    df_predictions["subject"]=subject

    metrics.append(df_metric)
    metrics_shuffle.append(df_metric_shuffle)
    predictions.append(df_predictions)
    
       
    return metrics,metrics_shuffle, predictions

### Apply Regression Models after Removing Voxels that have noise ceiling less than 0.3 / are zero or nan /applying functional mask


In [12]:
# noise_ceiling = pd.read_csv(f'{base_dir}/fmri/Fahime/single_trial/noise_ceiling/noise_ceiling_{subject}_{roi}.csv')
# noise_ceiling.rename(columns={'fmri':'nc'}, inplace=True)
# noise_ceiling_avg = noise_ceiling.groupby(['voxel','subject','roi']).mean()
#
# # noise_ceiling_avg= pd.merge(df,noise_ceiling_avg,on=['subject','roi','voxel'])
# noise_ceiling_avg.sort_values(by=['voxel','subject','roi'], inplace=True)
# noise_ceiling_avg['nc_selected']= noise_ceiling_avg['nc']>0.3

In [13]:
# noise_ceiling_avg

In [14]:
# model = 'ChemBERT_ChEMBL_pretrained'
# layer = 8
# subject =1
# roi = 'PirF'
# parent_input_sagar_original = f'{base_dir}/fmri/average_of_singletrial/fmri_{subject}_{roi}.csv'
# df = pd.read_csv(parent_input_sagar_original)
# fmri_array =df_to_cid_voxel_array(df)
# nc_mask =create_nc_mask( subject,roi)
# voxels_to_remove, masked_fmri =mask_fmri(fmri_array,nc_mask,subject,roi)


## Apply the regression model on the masked data with noise ceiling and functional mask

In [16]:
layers_start =[1,1,1,1,1000,1000]
layers_end = [9,7,13,13,1001,1001]
# layers_end = [1001]
# layers_end = [1001,1001]
 #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)

for m,model in enumerate([
    'ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1',
    'MoLFormer-XL-both-10pct',
                                     'SELFormer',
    'openpom',
    'behavior'
                                  ]):
    # for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
    for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
        for subject in [1,2,3]:

            for roi in ['PirF','PirT','AMY','OFC']:
                print(f"Processing subject {subject}, roi {roi}")

                parent_input_sagar_original = f'{base_dir}/fmri/average_of_singletrial/fmri_{subject}_{roi}.csv'
                df = pd.read_csv(parent_input_sagar_original)
                fmri_array =df_to_cid_voxel_array(df)
                nc_mask =create_nc_mask( subject,roi)
                _,fmri =mask_fmri(fmri_array,nc_mask,subject,roi)
                fmri = stats.zscore(fmri)

                print(f"Processingmodel {model}, layer {layer}")
                embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
                embeddings = stats.zscore(embeddings, axis=0)
                # embeddings_random=np.random.rand(embeddings.shape[0],embeddings.shape[1])
                metrics,metrics_shuffle, predictions = compute_correlation(embeddings,fmri,None,n_components=None)

                metrics = pd.concat(metrics)
                metrics_shuffle = pd.concat(metrics_shuffle)
                predictions =  pd.concat(predictions)
                predictions.to_csv(f"{base_dir}/fmri/Fahime/single_trial/metrics_functionalmasked_noiseceilingmasked_zscoredboth/subj_{subject}_{roi}_{model}_{layer}_predictions.csv",index=False)
                metrics.to_csv(f"{base_dir}/fmri/Fahime/single_trial/metrics_functionalmasked_noiseceilingmasked_zscoredboth/subj_{subject}_{roi}_{model}_{layer}_metrics.csv",index=False)
                metrics_shuffle.to_csv(f"{base_dir}/fmri/Fahime/single_trial/metrics_functionalmasked_noiseceilingmasked_zscoredboth/subj_{subject}_{roi}_{model}_{layer}_shuffle.csv",index=False)



Processing subject 1, roi PirF
(151,) shape
(151,) (151,) 1 PirF
(104, 96, 42) (151,)
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi PirT
(184,) shape
(184,) (184,) 1 PirT
(104, 96, 42) (184,)
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi AMY
(450,) shape
(450,) (450,) 1 AMY
(104, 96, 42) (450,)
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient 

Processing run 1


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 2


...

/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 13


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:27: NearConstantInputWarning: An input array is nearly constant; the computed corr

Processing run 14


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_14763/594800182.py:27: NearConstantInputWarning: An input array is nearly constant; the computed corr

## Apply the Regression Model on the average that we obtained!

In [17]:
layers_start =[1,1,1,1,1001,1001]
layers_end = [9,7,13,13,1001,1001]
# layers_end = [1001]
# layers_end = [1001,1001]
 #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)

for m,model in enumerate([
    'ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1',
    'MoLFormer-XL-both-10pct',
                                     'SELFormer',
    # 'openpom',
    # 'behavior'
                                  ]):
    # for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
    for layer in range(layers_start[m]-1,layers_start[m]-2,-1):
        for subject in [1,2,3]:

            for roi in ['PirF','PirT','AMY','OFC']:
                print(f"Processing subject {subject}, roi {roi}")

                # fmri, removed_voxels, retained_voxels = read_fmri_avgSingleTrial_peak(base_dir,subject,roi)

                # behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')

                fmri=read_orig_avg(base_dir, subject, roi)
                fmri = stats.zscore(fmri, axis=0)

                print(f"Processingmodel {model}, layer {layer}")
                embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
                embeddings = stats.zscore(embeddings, axis=0)
                # embeddings_random=np.random.rand(embeddings.shape[0],embeddings.shape[1])
                metrics,metrics_shuffle, predictions = compute_correlation(embeddings,fmri,None,n_components=None)

                metrics = pd.concat(metrics)
                metrics_shuffle = pd.concat(metrics_shuffle)
                predictions =  pd.concat(predictions)
                predictions.to_csv(f"{base_dir}/fmri/fmri_orig/subj_{subject}_{roi}_{model}_{layer}_predictions.csv",index=False)
                metrics.to_csv(f"{base_dir}/fmri/fmri_orig/subj_{subject}_{roi}_{model}_{layer}_metrics.csv",index=False)
                metrics_shuffle.to_csv(f"{base_dir}/fmri/fmri_orig/subj_{subject}_{roi}_{model}_{layer}_shuffle.csv",index=False)



Processing subject 1, roi PirF
sssss (160, 101)
Processingmodel ChemBERT_ChEMBL_pretrained, layer 0
Processing run 0


ValueError: 
All the 80 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/farzaneh/opt/anaconda3/envs/MoLFormer_fMRI/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/farzaneh/opt/anaconda3/envs/MoLFormer_fMRI/lib/python3.11/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/farzaneh/opt/anaconda3/envs/MoLFormer_fMRI/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py", line 1242, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Use

In [20]:
layers_start =[1,1,1,1,1000,1000]
layers_end = [9,7,13,13,1001,1001]
 #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)

for m,model in enumerate([
    'ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1','MoLFormer-XL-both-10pct',
                                     'SELFormer','behavior','openpom'
                                  ]):
    # for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
    for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
        for subject in [1,2,3]:

            for roi in ['PirF','PirT','AMY','OFC']:
                print(f"Processing subject {subject}, roi {roi}")

                fmri = read_fmri_avgSingleTrial_peak(base_dir,subject,roi)

                # behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')

                print(f"Processingmodel {model}, layer {layer}")
                embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
                # embeddings_random=np.random.rand(embeddings.shape[0],embeddings.shape[1])
                metrics,metrics_shuffle, predictions = compute_correlation(embeddings,fmri,n_components=None)
                # if metrics is None:
                #     continue
                metrics = pd.concat(metrics)
                metrics_shuffle = pd.concat(metrics_shuffle)
                predictions =  pd.concat(predictions)
                predictions.to_csv(f"{base_dir}/fmri/Fahime/single_trial/metrics/subj_{subject}_{roi}_{model}_{layer}_predictions.csv",index=False)
                metrics.to_csv(f"{base_dir}/fmri/Fahime/single_trial/metrics/subj_{subject}_{roi}_{model}_{layer}_metrics.csv",index=False)
                metrics_shuffle.to_csv(f"{base_dir}/fmri/Fahime/single_trial/metrics/subj_{subject}_{roi}_{model}_{layer}_shuffle.csv",index=False)



Processing subject 1, roi PirF
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi PirT
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi AMY
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processi

/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 1


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 2


...

/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 13


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 14


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_96233/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

In [9]:
layers_start =[1,1,1,1,1000,1000]
layers_end = [9,7,13,13,1001,1001]
 #rw<masks>nii = masks for PirF (rw.APC,nii), PirT (rw.PPC.nii), AMY (rwAmygdala.nii), OFC (rwOFC.nii)

for m,model in enumerate([
    'ChemBERT_ChEMBL_pretrained', 'ChemBERTa-zinc-base-v1','MoLFormer-XL-both-10pct',
                                     'SELFormer','behavior','openpom'
                                  ]):
    # for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
    for layer in range(layers_end[m]-1,layers_end[m]-2,-1):
        for subject in [1,2,3]:

            for roi in ['PirF','PirT','AMY','OFC']:
                print(f"Processing subject {subject}, roi {roi}")

                fmri = read_fmri_avgSingleTrial_peak(base_dir,subject,roi)

                # behavior_ratings =  np.load(f'../results/ratings_{subject}.npy')

                print(f"Processingmodel {model}, layer {layer}")
                embeddings = np.load(f'../results/embeddings_{model}_{subject}_{layer}.npy')
                # embeddings_random=np.random.rand(embeddings.shape[0],embeddings.shape[1])
                metrics,metrics_shuffle, predictions = compute_correlation(embeddings,fmri,n_components=None,learning_type='ridge')
                if metrics is None:
                    continue
                metrics = pd.concat(metrics)
                metrics_shuffle = pd.concat(metrics_shuffle)
                predictions =  pd.concat(predictions)
                predictions.to_csv(f"{base_dir}/fmri/average_of_singletrial/metrics/subj_{subject}_{roi}_{model}_{layer}_predictions.csv",index=False)
                metrics.to_csv(f"{base_dir}/fmri/average_of_singletrial/metrics/subj_{subject}_{roi}_{model}_{layer}_metrics.csv",index=False)
                metrics_shuffle.to_csv(f"{base_dir}/fmri/average_of_singletrial/metrics/subj_{subject}_{roi}_{model}_{layer}_shuffle.csv",index=False)



Processing subject 1, roi PirF
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi PirT
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processing run 13
Processing run 14
Processing subject 1, roi AMY
Processingmodel ChemBERT_ChEMBL_pretrained, layer 8
Processing run 0
Processing run 1
Processing run 2
Processing run 3
Processing run 4
Processing run 5
Processing run 6
Processing run 7
Processing run 8
Processing run 9
Processing run 10
Processing run 11
Processing run 12
Processi

/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 1


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 2


...

Processing run 13


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

Processing run 14


/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:22: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_correlations = [scipy.stats.spearmanr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:27: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlations_shuffle = [scipy.stats.pearsonr(predicted[:, i], y[:, i]) for i in range(y.shape[1])]
/var/folders/s8/bznqh0v13ddg31lz919tkrmc0000gn/T/ipykernel_79023/594800182.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.

OSError: Cannot save file into a non-existent directory: '../../../../T5 EVO/fmri/average_of_singletrial/metrics_cvridge'